In [1]:
import numpy as np
import pandas as pd
from pandas import json_normalize
import seaborn as sns
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup
import requests 
import time
import re

# pip install geopy
from geopy.geocoders import Nominatim

import warnings
warnings.filterwarnings("ignore")

from geopy_functions import *
from my_functions import *

**YOB** te 5 albums a Sputnik, però nomes **2** a Discogs de UK, tot i que és una banda de UK. 

Si apareixen al df de US, hauré de fer un concat/merge

# **Locations Wikipedia scraping**

In [2]:
df_ratings = pd.read_csv('Datasets/df_ratings_top.csv', keep_default_na=False)

print(f"{df_ratings['artist'].nunique()} artists")
print(f"{df_ratings.shape[0]} albums")
print(f"Average of {round(df_ratings.shape[0] / df_ratings['artist'].nunique(), 2)} albums per artist in the df that has all the albums with any ratings")

23683 artists
79625 albums
Average of 3.36 albums per artist in the df that has all the albums with any ratings


In [53]:
df = pd.read_csv('Datasets/df_rock_ratings.csv', keep_default_na=False)

print(f"{df['artist'].nunique()} artists")
print(f"{df.shape[0]} albums")
print(f"Average of {round(df.shape[0] / df['artist'].nunique(), 2)} albums per artist in the subset with the (mostly UK) albums with more than 10 votes")

5094 artists
12059 albums
Average of 2.37 albums per artist in the subset with the (mostly UK) albums with more than 10 votes


## Testing

In [28]:
df.sample(5)

,artist,title,year,album_length,tracks,genres,styles,release_country,artist_profile
10580,The 1975,A Brief Inquiry into Online Relationships,2018,58.43,15,"['Rock', 'Pop']","['Indie Rock', 'Alternative Rock', 'Indie Pop']",UK & Europe,British indie rock band. \r\n\r\nPop-rock band...
9889,Le Butcherettes,A Raw Youth,2015,0.00,12,['Rock'],[],US,Formed by Teri Gender Bender and Auryn Jolene ...
6394,John Fogerty,Centerfield,1985,35.33,9,['Rock'],"['Pop Rock', 'Folk Rock', 'Country Rock']",US,"American musician, songwriter, and guitarist (..."
4498,L7,The Beauty Process: Triple Platinum,1997,41.57,12,['Rock'],"['Punk', 'Grunge']",US,American grunge punk/alternative rock band fro...
5742,The Fall,Are You Are Missing Winner,2001,47.68,10,['Rock'],"['Garage Rock', 'Punk', 'Rockabilly']",UK,"Post-punk band from Greater Manchester, UK. 19..."


In [125]:
artists = df['artist'].unique()
len(artists)

9616

In [106]:
artists[4155]

'Life at These Speeds'

In [116]:
artist = "AABSINTHE"
name_changed = artist.title().replace(' ', '_')
name_changed

'Aabsinthe'

In [658]:
artist = "John Grant"
name_changed = artist.replace(' ', '_')
name_changed_band = artist.replace(' ', '_') + ('_(band)')
name_changed_musician = name_changed + ('_(musician)')

url = f"https://en.wikipedia.org/wiki/{name_changed_musician}"
response = requests.get(url).content
soup = BeautifulSoup(response, "html.parser")
table = soup.select('#mw-content-text > div.mw-content-ltr.mw-parser-output > table.infobox')

try:
    text = table[0].text

    # Step 1: Extract the part after 'Born'
    after_born = text.split("Born", 1)[1]

    text_age = re.search("aged", after_born)

    if text_age:
        # This means the musician is dead
        location = re.split(r'(19\d{2})', after_born)[4].split('Died')[0].strip()
    else:
        try:
            text = re.split(r'(19\d{2})', after_born)[4].split(')')[1]

            if "Other\xa0names" in text:
                location = text.split('Other\xa0names')[0]
            else:
                if "Citizenship" in text:
                    location = text.split('Citizenship')[0]
                else:
                    if "Genres" in text:
                        location = text.split('Genres')[0]
                        print('Genres')
                    else:
                        if "Occupations" in text:
                            location = text.split('Occupations')[0]
                            print('Occupations')
                        else:
                            location = np.nan
        except:  
            location = np.nan
except:
    print('fuck')


Genres


In [659]:
location

'Buchanan, Michigan, U.S.'

In [652]:
name_changed_musician

'John_Grant_(musician)'

In [533]:
artist = "Stone Gossard"
name_changed = artist.replace(' ', '_')
name_changed_band = artist.replace(' ', '_') + ('_(band)')

url = f"https://en.wikipedia.org/wiki/{name_changed}"
response = requests.get(url).content
soup = BeautifulSoup(response, "html.parser")
table = soup.select('#mw-content-text > div.mw-content-ltr.mw-parser-output > table.infobox')

try:
    location = table[0].text.split('Origin')[1].split('Genres')[0]

# save info in lists
    print('origin')

except:
    text = table[0].text

    # Step 1: Extract the part after 'Born'
    after_born = text.split("Born", 1)[1]

    text_age = re.search("aged", after_born)

    if text_age:
        # This means the artist is dead
        print('dead')
        location = re.split(r'(19\d{2})', after_born)[4].split('Died')[0].strip()
    else:
        try:
            text = re.split(r'(19\d{2})', after_born)[4].split(')')[1]

            if "Other\xa0names" in text:
                location = text.split('Other\xa0names')[0]
            else:
                if "Citizenship" in text:
                    location = text.split('Citizenship')[0]
                else:
                    if "Occupations" in text:
                        location = text.split('Occupations')[0]
                    else:
                        if "Genres" in text:
                            location = text.split('Genres')[0]
                            print(repr(location))
                        else:
                            location = np.nan
        except:  
            location = np.nan


In [535]:
text

'Seattle, Washington, U.S.GenresAlternative rockgrungeglam punkpunk rockhard rockheavy metalglam metalOccupationsMusiciansongwriterInstrumentsGuitarvocalsYears active'

In [536]:
location

'Seattle, Washington, U.S.GenresAlternative rockgrungeglam punkpunk rockhard rockheavy metalglam metal'

In [118]:
df.head()

,album_id,artist,album,rating,votes
0,37,Less Than Jake,Losing Streak,3.90,414
1,40,Sparta,Wiretap Scars,3.79,431
2,41,Muse,Absolution,3.99,4411
3,42,Muse,Showbiz,3.50,2181
4,45,Finch,What It Is to Burn,3.69,864


## **Changing values**

In [49]:
df

,year,artist,title,rating,votes,album_length,tracks,styles,release_country,artist_profile
0,2002,Sparta,Wiretap Scars,3.79,431,47.62,12,"['Indie Rock', 'Alternative Rock']",US,"El Paso, Texas based band. Formed in 2001 afte..."
1,2003,Muse,Absolution,3.99,4411,52.18,14,['Alternative Rock'],Europe,"Muse are an English rock band from Teignmouth,..."
2,1999,Muse,Showbiz,3.50,2181,49.65,12,"['Alternative Rock', 'Prog Rock']",UK,"Muse are an English rock band from Teignmouth,..."
3,2002,Finch,What It Is to Burn,3.69,864,61.18,13,"['Emo', 'Post-Hardcore', 'Alternative Rock']",US,"Finch is a post-hardcore band from Temecula, C..."
4,2002,Transplants,Transplants,3.35,198,45.23,12,['Punk'],US,"Punk rock/rap rock band Los Angeles, CA, Unite..."
...,...,...,...,...,...,...,...,...,...,...
12054,2024,As Living Arrows,Hope and Ruin,3.80,47,0.00,8,['Post-Hardcore'],NaN,"Post-screamo band from Brighton, UK\r\n"
12055,2024,The Dopamines,80/20,3.92,13,0.00,13,"['Punk', 'Pop Punk']",US,"Punk band from Cincinnati, Ohio."
12056,2024,Dawnwalker,The Unknowing,3.68,11,0.00,9,['Prog Rock'],UK,"Post-Metal/Rock band from London, UK."
12057,2024,Tiny Moving Parts,Deep in the Blue,3.02,28,0.00,10,"['Emo', 'Math Rock', 'Pop Punk']",US,"Trio from Benson, Minnesota. The band takes in..."


In [579]:
df[df['artist']=='Nucleus']

,year,artist,title,rating,votes,album_length,tracks,styles,release_country,artist_profile


In [580]:
df[df['artist'].str.contains('Nucleus')]

,year,artist,title,rating,votes,album_length,tracks,styles,release_country,artist_profile
7537,1970,Nucleus (UK),Elastic Rock,3.55,20,0.00,13,"['Jazz-Rock', 'Fusion', 'Prog Rock']",UK,"Pioneering jazz-rock, progressive, psychedelic..."
7568,1971,Nucleus (UK),We'll Talk About It Later,3.79,21,0.00,7,"['Fusion', 'Jazz-Funk', 'Jazz-Rock', 'Prog Rock']",UK,"Pioneering jazz-rock, progressive, psychedelic..."
9833,2016,Nucleus (US),Sentient,3.29,29,37.92,9,['Death Metal'],US,"Death Metal band from Chicago, Illinois, USA. ..."
10933,2019,Nucleus (US),Entity,3.68,40,38.42,8,['Death Metal'],US,"Death Metal band from Chicago, Illinois, USA. ..."


In [578]:
df.loc[7537, 'artist'] = 'Nucleus (UK)'
df.loc[7568, 'artist'] = 'Nucleus (UK)'
df.loc[9833, 'artist'] = 'Nucleus (US)'
df.loc[10933, 'artist'] = 'Nucleus (US)'

In [869]:
df.drop([7660, 8037], axis=0, inplace=True)
df.shape

(12059, 10)

In [122]:
df['artist'] = np.where(df['artist']=='pg.99 / Majority Rule', 'Majority Rule', df['artist'])
df[df['artist']=='pg.99 / Majority Rule']

,album_id,artist,album,rating,votes


In [582]:
df.to_csv('Datasets/df_rock_ratings.csv', index=False)

In [581]:
df.shape

(12059, 10)

### **Testing code for strange cases**

In [ ]:
df = pd.read_csv('Datasets/df_ratings.csv')
artists = df['artist'].unique()

artists_list = []
origin_list = []
count=0

for index in artists[140:150]:

    artists_list.append(index)
    name_changed = index.replace(' ', '_')
    name_changed_band = name_changed + ('_(band)')

    try:
        url = f"https://en.wikipedia.org/wiki/{name_changed_band}"
        response = requests.get(url).content
        soup = BeautifulSoup(response, "html.parser")

        origin = soup.select('table tr th', class_='infobox-label')

        if len(origin) > 0:
            try:
                if origin[2].text == 'Origin':
                    location = soup.select('table tr td', class_='infobox-data')[1].text
                elif origin[3].text == 'Origin':
                    location = soup.select('table tr td', class_='infobox-data')[2].text
                # else:
                    
                if 'multiple issues' in location:
                    location = soup.select('table tr td', class_='infobox-data')[7].text        
                    print(f'{count} - {name_changed_band}: multiple issues - {location}')
                    origin_list.append(location)
                elif 'additional citations' in location:
                    location = soup.select('table tr td', class_='infobox-data')[3].text        
                    print(f'{count} - {name_changed_band}: additional citations - {location}')
                    origin_list.append(location)

                else:
                    print(f'{count} - {name_changed_band}: {location}')
                    origin_list.append(location)
            except:
                print(f'{count} - {name_changed_band}: {location}')
                origin_list.append(location)      
        else:
            try:
                url = f"https://en.wikipedia.org/wiki/{name_changed}"
                response = requests.get(url).content
                soup = BeautifulSoup(response, "html.parser")

                origin = soup.select('table tr th', class_='infobox-label')

                if len(origin) > 0:
                    if origin[2].text == 'Origin':
                        location = soup.select('table tr td', class_='infobox-data')[1].text

                        if 'multiple issues' in location:
                            location = soup.select('table tr td', class_='infobox-data')[7].text        
                            print(f'{count} - {name_changed_band}: multiple issues - {location}')
                            origin_list.append(location)
                        elif 'additional citations' in location:
                            location = soup.select('table tr td', class_='infobox-data')[3].text        
                            print(f'{count} - {name_changed_band}: additional citations - {location}')
                            origin_list.append(location)
                        else:
                            print(f'{count} - {name_changed_band}: {location}')
                            origin_list.append(location)

                    elif origin[3].text == 'Origin':
                        location = soup.select('table tr td', class_='infobox-data')[2].text
                        print(f'{count} - {name_changed_band}: {location}')
                        origin_list.append(location) 

                    else:
                        print(f'{count} - {index}: no location found')
                        origin_list.append(np.nan)  
                else:
                    print(f'{count} - {index}: short length')
                    origin_list.append(np.nan)
            except:
                print(f'{count} - {index}: error')
                origin_list.append(np.nan)
    except:
        print(f'{count} - {index}: error')
        origin_list.append(np.nan)

    if len(artists_list) != len(origin_list):
        print('different lengths')
        break

    count+=1

0 - Funeral_for_a_Friend_(band): Bridgend, Wales
1 - Millencolin_(band): multiple issues - Örebro, Sweden
2 - The_Flaming_Lips_(band): Oklahoma City, Oklahoma, U.S.
3 - Feeder_(band): Feeder in 2008
4 - Descendents_(band): Manhattan Beach, California, U.S.
5 - PJ Harvey: no location found
6 - Godsmack_(band): Lawrence, Massachusetts U.S.
7 - Blind_Faith_(band): Ripley, Surrey, England
8 - Van_Halen_(band): Pasadena, California, U.S.
9 - Damageplan_(band): Dallas, Texas, U.S.


## Defining the functions

In [ ]:
def get_origins_wikipedia(df, start_index, final_index):
    df = pd.read_csv('Datasets/df_rock_ratings.csv')
    artists = df['artist'].unique()

    try:
    # import the DataFrame with the locations whose coordinates I already have
        df_coordinates_scraped = pd.read_csv('Datasets/df_coordinates.csv')
        print('Bingo! df_coordinates.csv found \n')
    except: 
        print('df_coordinates.csv not found \n')

    artists_list = []
    origin_list = []
    count=0
    scraped=0

    for index in artists[start_index:final_index]:

        name_changed = index.replace(' ', '_')
        name_changed_band = name_changed + ('_(band)')

        try:
            url = f"https://en.wikipedia.org/wiki/{name_changed_band}"
            response = requests.get(url).content
            soup = BeautifulSoup(response, "html.parser")

            table = soup.select('#mw-content-text > div.mw-content-ltr.mw-parser-output > table.infobox')

            location = table[0].text.split('Origin')[1].split('Genres')[0]
            city = location.split(', ')[0]
            count+=1
            
        # save info in lists
            artists_list.append(index)  
            origin_list.append(location)
            scraped+=1
            print(f'{scraped}/{count} - {name_changed_band}: {location}')

        except:
            try:
                url = f"https://en.wikipedia.org/wiki/{name_changed}"
                response = requests.get(url).content
                soup = BeautifulSoup(response, "html.parser")
                table = soup.select('#mw-content-text > div.mw-content-ltr.mw-parser-output > table.infobox')

                try:
                    location = table[0].text.split('Origin')[1].split('Genres')[0]
                    city = location.split(', ')[0]
                    count+=1 
    
                # save info in lists
                    artists_list.append(index)  
                    origin_list.append(location)
                    scraped+=1
                    print(f'{scraped}/{count} - {name_changed}: {location}')

                except:
                    location = table[0].text.split(')')[2].split('Genres')[0]
                    city = location.split(', ')[0]
                    count+=1

                # save info in lists
                    artists_list.append(index)  
                    origin_list.append(location)
                    scraped+=1
                    print(f'{scraped}/{count} - {name_changed} (individual): {location}')

            except:
                try:
                    url = f"https://es.wikipedia.org/wiki/{name_changed}"
                    response = requests.get(url).content
                    soup = BeautifulSoup(response, "html.parser")

                    table = soup.select('#mw-content-text > div.mw-content-ltr.mw-parser-output > table.infobox')
                    location = table[0].text.split('Origen\n')[1].split(' Información')[0]
                    city = location.split(', ')[0]
                    count+=1    
    
                # save info in lists
                    artists_list.append(index)  
                    origin_list.append(location)
                    scraped+=1
                    print(f'{scraped}/{count} - {name_changed} (español): {location}')

                except:
                    scraped+=1
                    print(f'{scraped}/{count} - {index}: error')
                    artists_list.append(index) 
                    origin_list.append(np.nan)

        if len(artists_list) != len(origin_list):
            print('different lengths')
            break

    df_artists_origins = pd.DataFrame({'artist': artists_list
                             , 'origin': origin_list})
    
    return df_artists_origins

In [102]:
def get_new_artists(df_artists_origins):

# import the df with the artists' origins already scraped
    df_artists_origins_scraped = pd.read_csv('Datasets/df_artists_origins.csv')

    if df_artists_origins['origin'].isna().sum() == 0:        
        print("No null values, but let's take a look just in case there are weird locations")

    else: 
    # take a look at the df with the new artists and make sure there are non null values in origin (when it couldn't find it in Wikipedia)
        print(f'{round(df_artists_origins['origin'].isna().sum() / df_artists_origins.shape[0]*100, 2)} % of nulls')
    
# subset of the new artists I just got, wether there are null values or not
    df_new_artists = df_artists_origins[~df_artists_origins['artist'].isin(df_artists_origins_scraped['artist'].values)]

    print("Here is the dataframe with the new artists, without duplicates")
    return df_new_artists   # so I can take a look at it and then continue

In [ ]:
def export_artists_origins_concat(df_new_artists):

# import the df with the artists' origins already scraped
    df_artists_origins_scraped = pd.read_csv('Datasets/df_artists_origins.csv')

# concat with the df I just got
    df_artists_origins_concat = pd.concat([df_artists_origins_scraped, df_new_artists])
    df_artists_origins_concat.drop_duplicates(inplace=True)     # just in case
    df_artists_origins_concat.reset_index(drop=True, inplace=True)

# export all the artists and their origins to a .csv file (the ones I got plus the new artists)
    df_artists_origins_concat.to_csv('Datasets/df_artists_origins.csv', index=False)
    print('df_artists_origins_concat exported to .csv')
    print(df_artists_origins_concat.shape)

In [175]:
def get_coordinates_geopy(df_new_artists):
    
# replace special characters for spaces
    df_new_artists['origin_clean'] = df_new_artists['origin'].str.replace('.', '')
    df_new_artists['origin_clean'] = df_new_artists['origin_clean'].str.replace(r'\[\d+\]', '', regex=True)

# run the function that gets the coordinates from the origins from Geopy
    geolocator = Nominatim(user_agent="music_analysis", timeout=10)

# if they are 'dirty' origins that after the cleaning, they result in the same 'origin_clean'
    df_unique = df_new_artists[['origin', 'origin_clean']].drop_duplicates() 
    unique_origins = df_unique['origin'].values
    unique_origins_clean = df_unique['origin_clean'].values

    country_list = []
    city_list = []
    latitude_list = []
    longitude_list = []
    address_list = []
    lists = [country_list, city_list, latitude_list, longitude_list, address_list]
    count = 0

    for origin in unique_origins_clean:
        count+=1
        time.sleep(1)
        location = geolocator.geocode(origin)

        print(f'{count}/{len(unique_origins_clean)} - {location.address}')  

    # save the info in lists
        country_list.append(location.address.split(', ')[-1])
        city_list.append(origin.split(', ')[0])
        latitude_list.append(location.latitude)
        longitude_list.append(location.longitude)
        address_list.append(location.address)

        # # Check lengths
        # print(f"{count}/{len(unique_origins_clean)} - {origin}")
        # print(f"Current list lengths -> country: {len(country_list)}, city: {len(city_list)}, "
        #     f"lat: {len(latitude_list)}, lon: {len(longitude_list)}, address: {len(address_list)}")

    df_coordinates = pd.DataFrame({'country': country_list
                                , 'city': city_list
                                , 'origin': unique_origins
                                , 'origin_clean': unique_origins_clean
                                , 'latitude': latitude_list
                                , 'longitude': longitude_list
                                , 'address': address_list})
    df_coordinates.sort_values(['country', 'city'], inplace=True) # sort by country and city
    df_coordinates.reset_index(drop=True, inplace=True)

    return df_coordinates

In [ ]:
def export_coordinates_concat(df_coordinates):

# import the last df that contains the coordinates of the unique origins
    df_coordinates_scraped = pd.read_csv('Datasets/df_coordinates.csv')
    print(f"df_coordinates_scraped: {df_coordinates_scraped.shape}\n")

# concat with the df of the coordinates I just got
    df_coordinates_concat = pd.concat([df_coordinates_scraped, df_coordinates])
    df_coordinates_concat.sort_values(['country', 'city'], inplace=True) # sort by country and city
    df_coordinates_concat.reset_index(drop=True, inplace=True)

# look for duplicates in the origin, between the locations I had already gotten and the new ones
    check_duplicates_origins(df_coordinates_concat)
    new_origins = df_coordinates_concat.shape[0] - df_coordinates_scraped.shape[0]
    print(f"Merged artists with coordinates! Found {new_origins} new locations")

# save it in a csv file (the coordinates I had plus the ones from the new artists I just got)
    df_coordinates_concat.to_csv('Datasets/df_coordinates.csv', index=False)
    print('df_coordinates_concat exported to .csv')

In [140]:
def merge_origins_coordinates(df_new_artists):

# import the last df that contains the coordinates of the unique origins
    df_coordinates_concat = pd.read_csv('Datasets/df_coordinates.csv')

# merge with the previous df with the artists
    df_artists_origins_coordinates = pd.merge(df_new_artists, df_coordinates_concat, on=['origin'])
    df_artists_origins_coordinates.drop(columns=['origin', 'origin_clean_x', 'origin_clean_y'], inplace=True)

# import the df that contains info of the artists and the coordinates of their origins
    df_artists_origins_coordinates_scraped = pd.read_csv('Datasets/df_artists_origins_coordinates.csv')

# concat to get the df with all the artists, origins and their coordinates
    df_artists_origins_coordinates_concat = pd.concat([df_artists_origins_coordinates_scraped, df_artists_origins_coordinates])
    df_artists_origins_coordinates_concat.reset_index(drop=True, inplace=True)

# save it in a csv file
    df_artists_origins_coordinates_concat.to_csv('Datasets/df_artists_origins_coordinates.csv', index=False)
    print("Exported to a .csv file")

    return df_artists_origins_coordinates_concat

## **Code to execute the functions from ``geopy_functions.py``**

In [11]:
# I have to import this df for the function to properly work
df = pd.read_csv('Datasets/df_rock_ratings.csv')
artists = df['artist'].unique()
len(artists)

5094

In [12]:
df_artists_origins = pd.read_csv('Datasets/df_artists_origins.csv')
df_artists_origins.index[-1]

2317

In [13]:
df_artists_origins.tail()

,artist,origin
2313,Broken Bells,"Los Angeles, California, U.S."
2314,New Kids on the Block,"Dorchester, Massachusetts, U.S."
2315,Joe Walsh,"Wichita, Kansas, U.S."
2316,City of Fire,"Vancouver, Canada"
2317,Creeper Lagoon,"San Francisco, California, United States"


In [14]:
np.where(artists=='Creeper Lagoon')

(array([2443], dtype=int64),)

In [15]:
artists[2443]

'Creeper Lagoon'

In [20]:
np.where(artists=='Boz Scaggs')

(array([3439], dtype=int64),)

In [21]:
artists[3439]

'Boz Scaggs'

In [16]:
# create the df with the origins scraped from Wikipedia

df_artists_origins = pd.read_csv('Datasets/df_artists_origins.csv')
start_index = 2440
final_index = start_index+1000

df_artists_origins = get_origins_wikipedia(df, start_index, final_index)

Bingo! df_coordinates.csv found 

1/1 - New_Kids_on_the_Block: Dorchester, Massachusetts, U.S.
2/2 - Joe_Walsh (individual): Wichita, Kansas, U.S.GenresHard rockblues rockpop rockprogressive rockcountry rock
3/3 - City_of_Fire_(band): Vancouver, Canada
4/4 - Creeper_Lagoon: San Francisco, California, United States
5/5 - Scanner_(band): Gelsenkirchen,  Germany
6/6 - The_Outfield: London
7/7 - Demonic_Resurrection: Mumbai, Maharashtra, India
8/8 - Between_the_Screams: Southend-on-Sea, Essex, England
8/9 - Flymore: error
9/10 - Get_Well_Soon_(band): Germany
10/11 - Take_That: Manchester, England
11/12 - Poster_Children: Champaign, Illinois, USA
12/13 - The_Soft_Pack: San Diego, United States
13/14 - Rickie_Lee_Jones (individual): Chicago, Illinois, United States
14/15 - We_Are_The_Ocean: Loughton, Essex, England
15/16 - Basia_Bulat: London, Ontario, Canada
16/17 - Calabrese_(band): Phoenix, Arizona, United States
17/18 - Soulwax_(band): Ghent, Belgium
18/19 - Oh_No_Ono: Aalborg, Denmark
1

In [17]:
# create a subset of the new artists I just got, and tell me if there are nulls
df_new_artists = get_new_artists(df_artists_origins)

# show the first new artists, if they were duplicates they have been dropped
df_new_artists.head()

172 nulls (17.2 %)


,artist,origin
4,Scanner,"Gelsenkirchen, Germany"
5,The Outfield,London
6,Demonic Resurrection,"Mumbai, Maharashtra, India"
7,Between the Screams,"Southend-on-Sea, Essex, England"
8,Flymore,NaN


In [22]:
df_new_artists.shape

(996, 2)

### If there are null or weird values

In [474]:
df_new_artists['origin'].isna().sum()

20

**Drop artists that are not from the UK or the US**

In [685]:
# if they are not brittish or american, I can drop them
try:
    df_new_artists.drop([991], axis=0, inplace=True)
except:
    print("row/s not found")

In [525]:
# check if there's info of the artist origin in the column 'artist_profile'
df.loc[8264]['artist_profile']

'UK doom/death band. Formed in 2001, disbanded in 2015. '

In [524]:
# look for the albums of the artist in the original df to check it's the correct artist
df[df['artist']=="Indesinence".strip()].sort_values('votes', ascending=False).head()

,year,artist,title,rating,votes,album_length,tracks,styles,release_country,artist_profile
8264,2012,Indesinence,Vessels of Light and Decay,3.95,11,62.17,7,"['Doom Metal', 'Death Metal']",Canada,"UK doom/death band. Formed in 2001, disbanded ..."


In [527]:
df_new_artists[df_new_artists['origin'].isna()].head()

,artist,origin


**``np.where`` to replace the values for the real origins**

In [ ]:
# looking in the internet for the real origins of these artists

df_new_artists['origin'] = np.where(df_new_artists['artist']=="Toy", "Brighton, England", df_new_artists['origin'])
df_new_artists['origin'] = np.where(df_new_artists['artist']=="Sting", "England", df_new_artists['origin'])
df_new_artists['origin'] = np.where(df_new_artists['artist']=="Sting", "England", df_new_artists['origin'])
df_new_artists['origin'] = np.where(df_new_artists['artist']=="Sting", "England", df_new_artists['origin'])
df_new_artists['origin'] = np.where(df_new_artists['artist']=="Sting", "England", df_new_artists['origin'])
df_new_artists['origin'] = np.where(df_new_artists['artist']=="Sting", "England", df_new_artists['origin'])
df_new_artists['origin'] = np.where(df_new_artists['artist']=="Sting", "England", df_new_artists['origin'])
df_new_artists['origin'] = np.where(df_new_artists['artist']=="Sting", "England", df_new_artists['origin'])

In [716]:
# check if there's info of the artist origin in the column 'artist_profile'
df.loc[6381]['artist_profile']

'British progressive rock band. A kind of seminal super-group Quiet Sun existed in two phases:the original band formed in 1969 who disbanded after Phil Manzanera joined Roxy Music, and then again in 1975 they reformed specially to record the music they had originally composed 5 years earlier.'

In [784]:
# look for the albums of the artist in the original df to check it's the correct artist
df[df['artist']=="Forest".strip()].sort_values('votes', ascending=False).head()

,year,artist,title,rating,votes,album_length,tracks,styles,release_country,artist_profile
7824,1978,Forest,Forest,3.6,10,32.0,8,"['AOR', 'Jazz-Funk', 'Soul']",US,US rock/funk band from the 70s


In [802]:
start_index = 41

end_index = start_index+5
df_new_artists[start_index:end_index]

,artist,origin
53,Grotus,"San Francisco, California, U.S."
54,Fang Island,"Providence, Rhode Island; Brooklyn, NY, US"
55,No Use for a Name,"Sunnyvale, California, U.S."
56,The Almighty,"Glasgow, Scotland"
57,Clan of Xymox,"Nijmegen, Netherlands"


**Drop artists that are not from the UK or the US**

In [791]:
# if they are not brittish or american, I can drop them
print(df_new_artists.shape)
df_new_artists.drop([877], axis=0, inplace=True)
print(df_new_artists.shape)

(936, 2)
(935, 2)


In [795]:
df_new_artists.loc[978]

artist                                                  Toy
origin    Brighton, Inglaterra,  Reino UnidoEstado\nActi...
Name: 978, dtype: object

**Check long and short origins, probably wrong**

In [798]:
# print abnormally long origins and visually check if they are correct
for index, row in df_new_artists.iterrows():
    if len(row['origin']) > 50:
        print(index, row['origin'])

402 Saltburn-by-the-Sea, North Yorkshire, United Kingdom


In [799]:
# print abnormally short origins and visually check if they are correct
for index, row in df_new_artists.iterrows():
    if len(row['origin']) < 10:
        print(index, row['origin'])

193 London
472 Chicago
665 London
904 New York


**Drop artists that are not from the UK or the US**

In [882]:
# if they are not brittish or american, I can drop them
print(df_new_artists.shape)
df_new_artists.drop([935], axis=0, inplace=True)
print(df_new_artists.shape)

(934, 2)
(933, 2)


**Individuals that didn't get the right origin in Wikipedia**

In [ ]:
# individuals that didn't get the right origin in Wikipedia
df_new_artists[df_new_artists['origin'].str.contains('Genres')]

,artist,origin


In [744]:
# individuals that didn't get the right origin in Wikipedia
df_new_artists[df_new_artists['origin'].str.contains(' and ')]

,artist,origin
425,Little Comets,"Tyne and Wear, England"
545,Heart in Hand,"Bournemouth, Dorset and Southampton, Hampshire..."
711,Deaf Havana,"Hunstanton and King's Lynn, Norfolk, England"
960,Tysondog,"Newcastle upon Tyne, Tyne and Wear, United Kin..."
965,Great Big Sea,"St. John's, Newfoundland and Labrador, Canada"


In [813]:
# bands from Cumbria, Geopy doesn't detect it
df_new_artists[df_new_artists['origin'].str.contains('Cumbria')]

,artist,origin
194,It Bites,"Egremont, Cumbria, England"


In [ ]:
df_new_artists['origin'] = df_new_artists['origin'].apply(lambda x: x.replace('Cumbria', 'Cumberland'))

In [831]:
# bands from Middlesex, Geopy doesn't detect it
df_new_artists[df_new_artists['origin'].str.contains('Middlesex')]

,artist,origin


In [ ]:
df_new_artists['origin'] = df_new_artists['origin'].apply(lambda x: x.replace(', Middlesex', ''))

In [878]:
# bands from Yorkshire, Geopy doesn't detect it
df_new_artists[df_new_artists['origin'].str.contains('West Riding of Yorkshire')]

,artist,origin


In [ ]:
df_new_artists['origin'] = df_new_artists['origin'].apply(lambda x: x.replace(', West Riding of Yorkshire', ''))

In [ ]:
# bands from Merseyside, Geopy doesn't detect it
df_new_artists[df_new_artists['origin'].str.contains('Merseyside')]

,artist,origin


In [ ]:
df_new_artists['origin'] = df_new_artists['origin'].apply(lambda x: x.replace(', Merseyside', ''))

In [ ]:
# bands from United Kingdom, wrong origin, poor level of detail
df_new_artists[df_new_artists['origin']==('United Kingdom')]

**``np.where`` to replace the values for the real origins**

In [889]:
# looking in the internet for the real origins of these artists

df_new_artists['origin'] = np.where(df_new_artists['artist']=="Cressida", "London, England", df_new_artists['origin'])
df_new_artists['origin'] = np.where(df_new_artists['artist']=="Big Deal", "London, England", df_new_artists['origin'])
df_new_artists['origin'] = np.where(df_new_artists['artist']=="Storm Corrosion", "Hemel Hempstead, Hertfordshire, United Kingdom", df_new_artists['origin'])
df_new_artists['origin'] = np.where(df_new_artists['artist']=="The Lumineers", "Denver, Colorado, U.S.", df_new_artists['origin'])
df_new_artists['origin'] = np.where(df_new_artists['artist']=="Allan Holdsworth", "Bradford, England", df_new_artists['origin'])
df_new_artists['origin'] = np.where(df_new_artists['artist']=="AxeWound", "London, England", df_new_artists['origin'])
df_new_artists['origin'] = np.where(df_new_artists['artist']=="We Came Out Like Tigers", "Liverpool, England", df_new_artists['origin'])
df_new_artists['origin'] = np.where(df_new_artists['artist']=="Paws", "Glasgow, Glasgow City, United Kingdom", df_new_artists['origin'])
df_new_artists['origin'] = np.where(df_new_artists['artist']=="Sting", "England", df_new_artists['origin'])
df_new_artists['origin'] = np.where(df_new_artists['artist']=="Sting", "England", df_new_artists['origin'])

**Try a single origin in Geopy**

In [885]:
# try to get the right origin of an origin that crashed
geolocator = Nominatim(user_agent="music_analysis")

origin = "Liverpool, Merseyside, United Kingdom"

origin_clean = re.sub(r'\[\d+\]', '', origin).replace('.', '')
location = geolocator.geocode(origin_clean)
print(f"{location.address}")
print(location.latitude, location.longitude)

AttributeError: 'NoneType' object has no attribute 'address'

In [890]:
start_index = 917

end_index = start_index+5
df_new_artists[start_index:end_index]

,artist,origin
984,Melody's Echo Chamber,"Aix-en-Provence, Bouches-du-Rhône, France"
985,Jonathan Wilson,"Laurel Canyon, Los Angeles"
986,Paws,"Glasgow, Glasgow City, United Kingdom"
987,Teen Daze,"Abbotsford, British Columbia, Canada"
988,Binah,"Guildford, England"


**Try all origins in Geopy**

In [891]:
# try to get the coordinates of the origins from Geopy and see if it crashes (wrong location that I have to change)
geolocator = Nominatim(user_agent="music_analysis", timeout=10)

initial_index = 917
count = initial_index-1

for origin in df_new_artists['origin'].str.replace('.', '').str.replace(r'\[\d+\]', '', regex=True)[initial_index:]:
    count+=1
    location = geolocator.geocode(origin)
    print(f"{count} - {location.address}")

917 - Aix-en-Provence, Bouches-du-Rhône, Provence-Alpes-Côte d'Azur, France métropolitaine, France
918 - Laurel Canyon, Chandler Boulevard, Garnsey, Valley Village Neighborhood Council District, Los Angeles, Los Angeles County, California, 91607, United States
919 - Glasgow, Glasgow City, Alba / Scotland, G2 1AL, United Kingdom
920 - Abbotsford, Fraser Valley Regional District, British Columbia, Canada
921 - Guildford, Surrey, England, United Kingdom
922 - Helsinki, Helsingin seutukunta, Uusimaa, Manner-Suomi, Suomi / Finland
923 - Los Angeles, Los Angeles County, California, United States
924 - Los Angeles, Los Angeles County, California, United States
925 - Hatfield, Welwyn Hatfield, Hertfordshire, England, United Kingdom
926 - Ottawa, Eastern Ontario, Ontario, Canada
927 - San Diego, San Diego County, California, United States
928 - London, Greater London, England, United Kingdom
929 - Hochschule München, 34, Lothstraße, Schwere-Reiter-Straße, Schwabing-West, München, Bayern, 80797,

#### Changing df_ratings in case needed

In [ ]:
# df_ratings['artist'] = np.where(df_ratings['artist']=='Incubus (USA-CA)', 'Incubus', df_ratings['artist'])
# df_ratings[df_ratings['artist']=='Incubus (USA-CA)']

,album_id,artist,album,rating,votes


In [ ]:
# df_ratings.to_csv('Datasets/df_ratings_top.csv', index=False)

## **Export to .csv**

### GeoPy wrong locations

**In case GeoPy fails due to a wrong location, I have to delete the new locations, export again, change the location and run GeoPy again**

In [ ]:
#     # import the df with the artists' origins already scraped
# df_artists_origins_scraped = pd.read_csv('Datasets/df_artists_origins.csv')
# df_artists_origins_scraped = df_artists_origins_scraped[0:-20]
# df_artists_origins_scraped.to_csv('Datasets/df_artists_origins.csv', index=False)

In [892]:
df_new_artists.shape

(933, 2)

In [90]:
# in case I created by mistake already 'origin_clean' and I want to drop it
# df_new_artists = df_new_artists[['artist', 'origin']]
# df_new_artists

### **Export to .csv**

In [893]:
export_artists_origins_concat(df_new_artists)

df_artists_origins_concat exported to .csv
(3251, 2)


## **GeoPy**

In [894]:
df_coordinates = get_coordinates_geopy(df_new_artists)
df_coordinates

1/580 - Gelsenkirchen, Nordrhein-Westfalen, Deutschland
2/580 - London, Greater London, England, United Kingdom
3/580 - Mumbai Suburban, Maharashtra, India
4/580 - Southend-on-Sea, Essex, England, United Kingdom
5/580 - Manchester, Greater Manchester, England, United Kingdom
6/580 - Champaign, Champaign County, Illinois, United States
7/580 - San Diego, San Diego County, California, United States
8/580 - Chicago, Cook County, Illinois, United States
9/580 - Loughton, Epping Forest, Essex, England, United Kingdom
10/580 - London, Southwestern Ontario, Ontario, Canada
11/580 - Phoenix, Maricopa County, Arizona, United States
12/580 - Gent, Oost-Vlaanderen, Vlaanderen, België / Belgique / Belgien
13/580 - Aalborg, Aalborg Kommune, Region Nordjylland, 9000, Danmark
14/580 - Warszawa, województwo mazowieckie, Polska
15/580 - Stockholm, Stockholms kommun, Stockholms län, 111 29, Sverige
16/580 - København, Københavns Kommune, Region Hovedstaden, 1357, Danmark
17/580 - Birmingham, West Midlan

,country,city,origin,origin_clean,latitude,longitude,address
0,Australia,Adelaide,"Adelaide, South Australia, Australia","Adelaide, South Australia, Australia",-34.928181,138.599931,"Adelaide, Adelaide City Council, South Austral..."
1,Australia,Blue Mountains,"Blue Mountains, NSW, Australia","Blue Mountains, NSW, Australia",-33.609741,150.405224,"Blue Mountains, New South Wales, Australia"
2,Australia,Brisbane,"Brisbane, Queensland, Australia","Brisbane, Queensland, Australia",-27.468968,153.023499,"City of Brisbane, Queensland, Australia"
3,Australia,Guildford,"Guildford, New South Wales, Australia","Guildford, New South Wales, Australia",-33.856105,150.993923,"Guildford, Sydney, Cumberland City Council, Ne..."
4,Australia,Lismore,"Lismore, New South Wales, Australia","Lismore, New South Wales, Australia",-28.807761,153.279188,"Lismore, Lismore City Council, New South Wales..."
...,...,...,...,...,...,...,...
575,日本,Kyoto,"Kyoto, Japan","Kyoto, Japan",35.021041,135.755607,"京都市, 京都府, 日本"
576,日本,Kōchi City,"Kōchi City, Japan","Kōchi City, Japan",34.673829,135.496971,"高知銀行, 四つ橋筋, 北堀江一丁目, 西区, 大阪市, 大阪府, 542-0086, 日本"
577,日本,Osaka,"Osaka, Japan","Osaka, Japan",34.693757,135.501454,"大阪市, 大阪府, 日本"
578,日本,Tokyo,"Tokyo, Japan","Tokyo, Japan",35.676860,139.763895,"東京都, 日本"


In [895]:
export_coordinates_concat(df_coordinates)

df_coordinates_scraped: (1144, 7)

Found 197 duplicates:
            city         country
2       Adelaide       Australia
5       Brisbane       Australia
15     Melbourne       Australia
16     Melbourne       Australia
22         Perth       Australia
...          ...             ...
1655  Washington   United States
1695      Dublin  Éire / Ireland
1711      Athens           Ελλάς
1720       Osaka              日本
1722       Tokyo              日本

[197 rows x 2 columns]

Resulting dataset: (1527, 7)
Merged artists with coordinates! Found 383 new locations
df_coordinates_concat exported to .csv


In [896]:
# merge with the dataframe containing all the artists and their origins
df_artists_origins_coordinates_concat = merge_origins_coordinates(df_new_artists)
df_artists_origins_coordinates_concat.tail()

Exported to a .csv file


,artist,country,city,latitude,longitude,address
3246,Indesinence,United Kingdom,London,51.489334,-0.144055,"London, Greater London, England, United Kingdom"
3247,Embryo,Deutschland,Munich,48.154141,11.556248,"Hochschule München, 34, Lothstraße, Schwere-Re..."
3248,Far East Family Band,日本,Toyohashi,34.769123,137.391461,"豊橋市, 愛知県, 日本"
3249,Boz Scaggs,United States,Plano,33.013676,-96.692510,"Plano, Collin County, Texas, United States"
3250,Nucleus (UK),United Kingdom,London,51.489334,-0.144055,"London, Greater London, England, United Kingdom"


## **Statistics**

In [897]:
brittish_cities = df_artists_origins_coordinates_concat[df_artists_origins_coordinates_concat['country']=='United Kingdom']
american_cities = df_artists_origins_coordinates_concat[df_artists_origins_coordinates_concat['country']=='United States']

In [898]:
order = df_artists_origins_coordinates_concat['country'].value_counts().index
print(df_artists_origins_coordinates_concat['country'].value_counts().head())

# plt.figure(figsize=(8,6))
# sns.countplot(df_artists_origins_coordinates_concat['country'], order=order)

country
United States     1595
United Kingdom    1126
Canada             101
Sverige             84
Australia           72
Name: count, dtype: int64


In [899]:
print(f"{brittish_cities.shape[0]} Brittish artists")
order = brittish_cities['city'].value_counts().index
print(f"{brittish_cities['city'].nunique()} Brittish cities\n")
print(brittish_cities['city'].value_counts().head(15))

# plt.figure(figsize=(9,45))
# sns.countplot(brittish_cities['city'], order=order)

1126 Brittish artists
372 Brittish cities

city
London            290
Glasgow            36
Birmingham         33
Manchester         27
Liverpool          23
Leeds              22
Brighton           18
United Kingdom     17
Bristol            16
Sheffield          14
Cardiff            11
Nottingham         11
Reading            10
Edinburgh          10
Essex               9
Name: count, dtype: int64


In [900]:
print(f"{american_cities.shape[0]} American artists")
order = american_cities['city'].value_counts().index
print(f"{american_cities['city'].nunique()} American cities\n")
print(american_cities['city'].value_counts().head(15))

# plt.figure(figsize=(5,55))
# sns.countplot(df_artists_origins_coordinates_concat[df_artists_origins_coordinates_concat['country']=='United States']['city'], order=order)

1595 American artists
494 American cities

city
Los Angeles      175
New York City    116
San Francisco     55
Chicago           53
Seattle           38
Brooklyn          36
Boston            28
Philadelphia      25
Portland          23
San Diego         21
Phoenix           18
Austin            18
Detroit           16
Washington        16
New York          15
Name: count, dtype: int64


# All separate codes without functions

## Actual code to get the origins from Wikipedia

In [42]:
artists_list, origin_list = get_origins_wikipedia(df, 20, 50)

Bingo! df_coordinates.csv found 

1/1 - The_New_Pornographers: Vancouver, British Columbia, Canada
2/2 - Regurgitator_(band): Brisbane, Queensland, Australia
3/3 - Sunny_Day_Real_Estate: Seattle, Washington, U.S.
4/4 - ...And_You_Will_Know_Us_by_the_Trail_of_Dead: Austin, Texas, U.S.
5/5 - Children_of_Bodom_(band): Espoo, Finland
6/6 - Audioslave: Glendale, California, U.S.
7/7 - Led_Zeppelin_(band): London, England
8/8 - Less_Than_Jake_(band): Gainesville, Florida, U.S.
9/9 - Alexisonfire: St. Catharines, Ontario, Canada
10/10 - Rammstein: Berlin, Germany
11/11 - Thrice_(band): Irvine, California, U.S.
12/12 - Lostprophets: Pontypridd, Wales
13/13 - New_Found_Glory_(band): Coral Springs, Florida, U.S.
14/14 - Placebo_(band): London, England
15/15 - Box_Car_Racer: San Diego, California, U.S.
16/16 - Thursday_(band): New Brunswick, New Jersey, U.S.
17/17 - Million_Dead: London, England
18/18 - The_Jesus_Lizard: Austin, Texas, U.S.
19/19 - The_Offspring_(band): Garden Grove, California, 

## Saving the data into a dataframe

In [43]:
df_artists_origins = pd.DataFrame({'artist': artists_list
                             , 'origin': origin_list})
df_artists_origins.shape

(30, 2)

In [44]:
df_artists_origins

,artist,origin
0,The New Pornographers,"Vancouver, British Columbia, Canada"
1,Regurgitator,"Brisbane, Queensland, Australia"
2,Sunny Day Real Estate,"Seattle, Washington, U.S."
3,...And You Will Know Us by the Trail of Dead,"Austin, Texas, U.S."
4,Children of Bodom,"Espoo, Finland"
5,Audioslave,"Glendale, California, U.S."
6,Led Zeppelin,"London, England"
7,Less Than Jake,"Gainesville, Florida, U.S."
8,Alexisonfire,"St. Catharines, Ontario, Canada"
9,Rammstein,"Berlin, Germany"


In [ ]:
# import the df with the artists' origins already scraped
df_artists_origins_scraped = pd.read_csv('Datasets/df_artists_origins.csv')
df_artists_origins_scraped.shape

(20, 2)

In [ ]:
# subset of the new artists I just got
df_new_artists = df_artists_origins[~df_artists_origins['artist'].isin(df_artists_origins_scraped['artist'].values)]
df_new_artists

,artist,origin
0,The New Pornographers,"Vancouver, British Columbia, Canada"
1,Regurgitator,"Brisbane, Queensland, Australia"
2,Sunny Day Real Estate,"Seattle, Washington, U.S."
3,...And You Will Know Us by the Trail of Dead,"Austin, Texas, U.S."
4,Children of Bodom,"Espoo, Finland"
5,Audioslave,"Glendale, California, U.S."
6,Led Zeppelin,"London, England"
7,Less Than Jake,"Gainesville, Florida, U.S."
8,Alexisonfire,"St. Catharines, Ontario, Canada"
9,Rammstein,"Berlin, Germany"


In [ ]:
# concat with the df I just got
df_artists_origins_concat = pd.concat([df_artists_origins_scraped, df_new_artists])
df_artists_origins_concat.reset_index(drop=True, inplace=True)
df_artists_origins_concat.shape

(50, 2)

I want to already export it to a .csv file so that later I can check the ``origin`` the next time I run the function and compare if I have scraped that origin already or not.

In [ ]:
# export all the artists and their origins to a .csv file (the ones I got plus the new artists)
df_artists_origins_concat.to_csv('Datasets/df_artists_origins.csv', index=False)

## ``origin`` cleaning: dots, [ ]...

The next thing I'm gonna do is cleaning the origin, because Geopy doesn't read correctly characters like "U.S.".

**Clean the ``origin`` column of all the artists, so I can look for the coordinates with Geopy**

In [50]:
df_new_artists.shape

(30, 2)

In [52]:
# replace special characters for spaces
df_new_artists['origin_clean'] = df_new_artists['origin'].str.replace('.', '')
df_new_artists['origin_clean'] = df_new_artists['origin_clean'].str.replace(r'\[\d+\]', '', regex=True)

# show the df to check it has been changed correctly
df_new_artists.tail()

,artist,origin,origin_clean
25,KISS,"New York City, U.S.","New York City, US"
26,The Polyphonic Spree,"Dallas, Texas, United States","Dallas, Texas, United States"
27,Cake,"Sacramento, California, U.S.","Sacramento, California, US"
28,Dinosaur Jr.,"Amherst, Massachusetts, U.S.","Amherst, Massachusetts, US"
29,U2,"Dublin, Ireland","Dublin, Ireland"


## Concat with the previous df

In [59]:
# import the last df that contains the coordinates of the unique origins
df_coordinates_scraped = pd.read_csv('Datasets/df_coordinates.csv')
df_coordinates_scraped.shape

(42, 7)

In [62]:
df_coordinates_scraped.head()

,country,city,origin,origin_clean,latitude,longitude,address
0,Australia,Brisbane,"Brisbane, Queensland, Australia","Brisbane, Queensland, Australia",-27.468968,153.023499,"City of Brisbane, Queensland, Australia"
1,Canada,St Catharines,"St. Catharines, Ontario, Canada","St Catharines, Ontario, Canada",43.157981,-79.244100,"St. Catharines, Niagara Region, Golden Horsesh..."
2,Canada,Vancouver,"Vancouver, British Columbia, Canada","Vancouver, British Columbia, Canada",49.260872,-123.113952,"Vancouver, Metro Vancouver Regional District, ..."
3,Deutschland,Berlin,"Berlin, Germany","Berlin, Germany",52.510885,13.398937,"Berlin, Pankow, Deutschland"
4,Suomi / Finland,Espoo,"Espoo, Finland","Espoo, Finland",60.204965,24.655981,"Espoo, Helsingin seutukunta, Uusimaa, Manner-S..."


In [ ]:
# concat with the df of the coordinates I just got
df_coordinates_concat = pd.concat([df_coordinates_scraped, df_coordinates])
df_coordinates_concat.sort_values(['country', 'city'], inplace=True) # sort by country and city
df_coordinates_concat.reset_index(drop=True, inplace=True)
df_coordinates_concat.shape

(43, 7)

In [59]:
# look for duplicates in the origin, between the locations I had already gotten and the new ones
check_duplicates_origins(df_coordinates_concat)

Found 1 duplicates:
     city         country
9  London  United Kingdom

Resulting dataset: (42, 7)


In [60]:
# save it in a csv file (the coordinates I had plus the ones from the new artists I just got)
df_coordinates_concat.to_csv('Datasets/df_coordinates.csv', index=False)

## **Merge ``origin`` with ``coordinates``**

In [66]:
df_new_artists

,artist,origin,origin_clean
0,The Clash,"London, England","London, England"
1,Darkthrone,"Kolbotn, Akershus, Norway","Kolbotn, Akershus, Norway"
2,Ramones,"Forest Hills, Queens, New York, U.S.","Forest Hills, Queens, New York, US"
3,Comets on Fire,"Santa Cruz, California, U.S.","Santa Cruz, California, US"
4,Fenix TX,"Houston, Texas, U.S.","Houston, Texas, US"
5,Incubus,"Calabasas, California, U.S.","Calabasas, California, US"
6,Feeder,"Newport, Wales","Newport, Wales"
7,PJ Harvey,"Bridport, Dorset, England","Bridport, Dorset, England"
8,Sebadoh,"Northampton, Massachusetts, U.S.","Northampton, Massachusetts, US"
9,Blind Faith,"Ripley, Surrey, England","Ripley, Surrey, England"


In [62]:
# merge with the previous df with the artists
df_artists_origins_coordinates = pd.merge(df_new_artists, df_coordinates_concat, on=['origin'])
df_artists_origins_coordinates.drop(columns=['origin', 'origin_clean_x', 'origin_clean_y'], inplace=True)

df_artists_origins_coordinates

,artist,country,city,latitude,longitude,address
0,The New Pornographers,Canada,Vancouver,49.260872,-123.113952,"Vancouver, Metro Vancouver Regional District, ..."
1,Regurgitator,Australia,Brisbane,-27.468968,153.023499,"City of Brisbane, Queensland, Australia"
2,Sunny Day Real Estate,United States,Seattle,47.603832,-122.330062,"Seattle, King County, Washington, United States"
3,...And You Will Know Us by the Trail of Dead,United States,Austin,30.271129,-97.743700,"Austin, Travis County, Texas, United States"
4,Children of Bodom,Suomi / Finland,Espoo,60.204965,24.655981,"Espoo, Helsingin seutukunta, Uusimaa, Manner-S..."
5,Audioslave,United States,Glendale,34.146942,-118.247847,"Glendale, Los Angeles County, California, Unit..."
6,Led Zeppelin,United Kingdom,London,51.489334,-0.144055,"London, Greater London, England, United Kingdom"
7,Less Than Jake,United States,Gainesville,29.651968,-82.324985,"Gainesville, Alachua County, Florida, United S..."
8,Alexisonfire,Canada,St Catharines,43.157981,-79.244100,"St. Catharines, Niagara Region, Golden Horsesh..."
9,Rammstein,Deutschland,Berlin,52.510885,13.398937,"Berlin, Pankow, Deutschland"


In [63]:
# locations of the new artists
df_artists_origins_coordinates['country'].value_counts()

country
United States      18
United Kingdom      5
Canada              2
Éire / Ireland      2
Australia           1
Suomi / Finland     1
Deutschland         1
Name: count, dtype: int64

In [ ]:
# import the df that contains info of the artists and the coordinates of their origins
df_artists_origins_coordinates_scraped = pd.read_csv('Datasets/df_artists_origins_coordinates.csv')
df_artists_origins_coordinates_scraped

,artist,country,city,latitude,longitude,address
0,Sparta,United States,El Paso,31.760116,-106.487040,"El Paso, El Paso County, Texas, United States"
1,Muse,United Kingdom,Teignmouth,50.546338,-3.495780,"Teignmouth, Teignbridge, Devon, England, Unite..."
2,Finch,United States,Temecula,33.494635,-117.147366,"Temecula, Riverside County, California, United..."
3,Transplants,United States,Los Angeles,34.053691,-118.242766,"Los Angeles, Los Angeles County, California, U..."
4,Rooney,United States,Los Angeles,34.053691,-118.242766,"Los Angeles, Los Angeles County, California, U..."
5,The Cure,United Kingdom,Crawley,51.110344,-0.180109,"Crawley, West Sussex, England, United Kingdom"
6,AFI,United States,Ukiah,39.150166,-123.207786,"Ukiah, Mendocino County, California, 95482, Un..."
7,Jimmy Eat World,United States,Mesa,33.415101,-111.831455,"Mesa, Maricopa County, Arizona, United States"
8,Black Sabbath,United Kingdom,Birmingham,52.479699,-1.902691,"Birmingham, West Midlands, England, United Kin..."
9,Yes,United Kingdom,London,51.489334,-0.144055,"London, Greater London, England, United Kingdom"


In [ ]:
df_artists_origins_coordinates_concat = pd.concat([df_artists_origins_coordinates_scraped, df_artists_origins_coordinates])
df_artists_origins_coordinates_concat.reset_index(drop=True, inplace=True)
df_artists_origins_coordinates_concat

,artist,country,city,latitude,longitude,address
0,Sparta,United States,El Paso,31.760116,-106.487040,"El Paso, El Paso County, Texas, United States"
1,Muse,United Kingdom,Teignmouth,50.546338,-3.495780,"Teignmouth, Teignbridge, Devon, England, Unite..."
2,Finch,United States,Temecula,33.494635,-117.147366,"Temecula, Riverside County, California, United..."
3,Transplants,United States,Los Angeles,34.053691,-118.242766,"Los Angeles, Los Angeles County, California, U..."
4,Rooney,United States,Los Angeles,34.053691,-118.242766,"Los Angeles, Los Angeles County, California, U..."
5,The Cure,United Kingdom,Crawley,51.110344,-0.180109,"Crawley, West Sussex, England, United Kingdom"
6,AFI,United States,Ukiah,39.150166,-123.207786,"Ukiah, Mendocino County, California, 95482, Un..."
7,Jimmy Eat World,United States,Mesa,33.415101,-111.831455,"Mesa, Maricopa County, Arizona, United States"
8,Black Sabbath,United Kingdom,Birmingham,52.479699,-1.902691,"Birmingham, West Midlands, England, United Kin..."
9,Yes,United Kingdom,London,51.489334,-0.144055,"London, Greater London, England, United Kingdom"


In [66]:
# save it in a csv file
df_artists_origins_coordinates_concat.to_csv('Datasets/df_artists_origins_coordinates.csv', index=False)

In [67]:
df_artists_origins_coordinates_concat.country.value_counts()

country
United States      30
United Kingdom     13
Canada              2
Éire / Ireland      2
Australia           1
Suomi / Finland     1
Deutschland         1
Name: count, dtype: int64